In [1]:
import tensorflow as tf
print(tf.__version__)

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv1D, Dense, Dropout, LayerNormalization, Bidirectional, LSTM, GRU, Layer, SpatialDropout1D, GlobalAveragePooling1D
from tensorflow.keras.layers import Lambda, Reshape, Flatten, Input, MultiHeadAttention, Flatten, Concatenate, Add, Multiply, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

2025-12-21 17:49:29.389536: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.20.0


In [2]:
import torch
import pandas as pd
import os, math, random
random.seed(2)
import numpy as np
import tensorflow as tf
import pyrsgis
from pyrsgis import raster
from osgeo import gdal
import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Lambda, Reshape
from scipy import interpolate
from tensorflow.keras.optimizers import Adam
import rasterio
import gc

In [3]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
# # For MCTNet

# class PositionalEncoding(layers.Layer):
#     """
#     Standard sinusoidal positional encoding
#     """
#     def __init__(self, d_model, **kwargs):
#         super().__init__(**kwargs)
#         self.d_model = d_model
        
#     def get_positional_encoding(self, time_steps, d_model):
#         """Generate sinusoidal positional encoding"""
#         positions = tf.range(time_steps, dtype=tf.float32)[:, tf.newaxis]
#         dims = tf.range(d_model, dtype=tf.float32)[tf.newaxis, :]
        
#         angle_rates = 1 / tf.pow(10000.0, (2 * (dims // 2)) / tf.cast(d_model, tf.float32))
#         angle_rads = positions * angle_rates
        
#         sines = tf.sin(angle_rads[:, 0::2])
#         cosines = tf.cos(angle_rads[:, 1::2])
        
#         pos_encoding = tf.concat([sines, cosines], axis=-1)
#         if d_model % 2 != 0:
#             pos_encoding = pos_encoding[:, :-1]
            
#         return pos_encoding
    
#     def call(self, inputs):
#         batch_size = tf.shape(inputs)[0]
#         time_steps = tf.shape(inputs)[1]
        
#         pos_encoding = self.get_positional_encoding(time_steps, self.d_model)
#         pos_encoding = tf.expand_dims(pos_encoding, 0)
#         pos_encoding = tf.tile(pos_encoding, [batch_size, 1, 1])
        
#         return inputs + pos_encoding


# class TransformerSubModule(layers.Layer):
#     """
#     Transformer sub-module with multi-head self-attention
#     """
#     def __init__(self, d_model, num_heads, dropout_rate=0.1, **kwargs):
#         super().__init__(**kwargs)
#         self.d_model = d_model
#         self.num_heads = num_heads
#         self.dropout_rate = dropout_rate
        
#     def build(self, input_shape):
#         # Multi-head attention
#         self.mha = layers.MultiHeadAttention(
#             num_heads=self.num_heads,
#             key_dim=self.d_model // self.num_heads,
#             dropout=self.dropout_rate
#         )
        
#         # Feed-forward network
#         self.dense1 = layers.Dense(self.d_model * 4, activation='relu')
#         self.dropout_ffn1 = layers.Dropout(self.dropout_rate)
#         self.dense2 = layers.Dense(self.d_model)
        
#         # Layer normalization
#         self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
#         self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        
#         # Dropout
#         self.dropout1 = layers.Dropout(self.dropout_rate)
#         self.dropout2 = layers.Dropout(self.dropout_rate)
        
#         super().build(input_shape)
    
#     def call(self, inputs, training=None):
#         # Multi-head self-attention
#         attn_output = self.mha(inputs, inputs, training=training)
#         attn_output = self.dropout1(attn_output, training=training)
#         out1 = self.layernorm1(inputs + attn_output)
        
#         # Feed-forward network
#         ffn_output = self.dense1(out1)
#         ffn_output = self.dropout_ffn1(ffn_output, training=training)
#         ffn_output = self.dense2(ffn_output)
#         ffn_output = self.dropout2(ffn_output, training=training)
#         out2 = self.layernorm2(out1 + ffn_output)
        
#         return out2


# class CNNSubModule(layers.Layer):
#     """
#     CNN sub-module with 1D convolutions along time dimension
#     """
#     def __init__(self, filters, kernel_size=3, dropout_rate=0.1, **kwargs):
#         super().__init__(**kwargs)
#         self.filters = filters
#         self.kernel_size = kernel_size
#         self.dropout_rate = dropout_rate
        
#     def build(self, input_shape):
#         input_dim = input_shape[-1]
        
#         # Projection layer if input dimension doesn't match filters
#         if input_dim != self.filters:
#             self.projection = layers.Dense(self.filters)
#         else:
#             self.projection = None
        
#         # Two 1D convolutional layers
#         self.conv1 = layers.Conv1D(
#             filters=self.filters,
#             kernel_size=self.kernel_size,
#             padding='same'
#         )
#         self.bn1 = layers.BatchNormalization()
        
#         self.conv2 = layers.Conv1D(
#             filters=self.filters,
#             kernel_size=self.kernel_size,
#             padding='same'
#         )
#         self.bn2 = layers.BatchNormalization()
        
#         self.dropout = layers.Dropout(self.dropout_rate)
        
#         super().build(input_shape)
    
#     def call(self, inputs, training=None):
#         # Project inputs if needed for residual connection
#         residual = inputs
#         if self.projection is not None:
#             residual = self.projection(residual)
        
#         # First conv block
#         x = self.conv1(inputs)
#         x = self.bn1(x, training=training)
#         x = tf.nn.relu(x)
        
#         # Second conv block
#         x = self.conv2(x)
#         x = self.bn2(x, training=training)
        
#         # Residual connection
#         x = x + residual
#         x = tf.nn.relu(x)
#         x = self.dropout(x, training=training)
        
#         return x


# class CTFusionModule(layers.Layer):
#     """
#     CNN-Transformer Fusion Module
#     Combines CNN and Transformer sub-modules in parallel
#     """
#     def __init__(self, d_model, num_heads, kernel_size=3, 
#                  dropout_rate=0.1, add_pos_encoding=False, **kwargs):
#         super().__init__(**kwargs)
#         self.d_model = d_model
#         self.num_heads = num_heads
#         self.kernel_size = kernel_size
#         self.dropout_rate = dropout_rate
#         self.add_pos_encoding = add_pos_encoding
        
#     def build(self, input_shape):
#         input_dim = input_shape[-1]
        
#         # Project input to d_model if needed
#         if input_dim != self.d_model:
#             self.input_projection = layers.Dense(self.d_model)
#         else:
#             self.input_projection = None
        
#         # Positional encoding (only for first stage)
#         if self.add_pos_encoding:
#             self.pos_encoding = PositionalEncoding(d_model=self.d_model)
        
#         # CNN sub-module
#         self.cnn_module = CNNSubModule(
#             filters=self.d_model,
#             kernel_size=self.kernel_size,
#             dropout_rate=self.dropout_rate
#         )
        
#         # Transformer sub-module
#         self.transformer_module = TransformerSubModule(
#             d_model=self.d_model,
#             num_heads=self.num_heads,
#             dropout_rate=self.dropout_rate
#         )
        
#         super().build(input_shape)
    
#     def call(self, inputs, training=None):
#         x = inputs
        
#         # Project to d_model if needed
#         if self.input_projection is not None:
#             x = self.input_projection(x)
        
#         # Add positional encoding in first stage
#         if self.add_pos_encoding:
#             x = self.pos_encoding(x)
        
#         # Parallel processing through CNN and Transformer
#         cnn_features = self.cnn_module(x, training=training)
#         transformer_features = self.transformer_module(x, training=training)
        
#         # Concatenate features from both branches
#         fused_features = layers.Concatenate(axis=-1)([cnn_features, transformer_features])
        
#         return fused_features



In [5]:
# model_path = '/home/jupyter-bryan/ISA_Data/ISA_Citarum_Multi_Orig_MCTnet.keras'
# model = load_model(model_path,
#                    custom_objects={'PositionalEncoding': PositionalEncoding,
#                                    'TransformerSubModule': TransformerSubModule,
#                                    'CNNSubModule': CNNSubModule,
#                                    'CTFusionModule': CTFusionModule},
#                    compile=False)

# Then recompile with your custom loss
# model.compile(
#     optimizer=Adam(learning_rate=0.001),
#     loss=MultiFocalLossSlopeElevationConstraint(),
#     metrics=['accuracy'])z

In [6]:
# @keras.saving.register_keras_serializable()
# def ReshapeLayer(x):
#     shape = x.shape
#     reshape = Reshape((shape[1],shape[2]))(x)
#     return reshape

In [7]:
# model_path = '/home/jupyter-bryan/ISA_Data/ISA_Citarum_Multi_Orig_1D_Wonosobo_FineTune.keras'
# model = load_model(model_path, custom_objects={'ReshapeLayer': ReshapeLayer}, compile=False)

In [8]:
model = load_model('/home/jupyter-bryan/ISA_Data/ISA_Citarum_Multi_Orig_1DConus_FineTune.keras')

I0000 00:00:1766310573.794507 2867471 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1089 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6


In [9]:
# # Load the saved model
# model_path = '/home/jupyter-bryan/ISA_Data/ISA_Citarum_Multi_Orig.keras'

# model = load_model(model_path,
#                    custom_objects={'MultiFocalLossSlopeElevationConstraint': MultiFocalLossSlopeElevationConstraint},
#                    compile=False)

# Load a new multispectral image
ds_ndvi, img_ndvi  = raster.read('/home/jupyter-bryan/ISA_Data/Raster/PH_3-Month_NDVI_2019_2024.tif')
ds_mndwi, img_mndwi  = raster.read('/home/jupyter-bryan/ISA_Data/Raster/PH_3-Month_MNDWI_2019_2024.tif')
ds_ndbi, img_ndbi  = raster.read('/home/jupyter-bryan/ISA_Data/Raster/PH_3-Month_NDBI_2019_2024.tif')
ds_ndbsi, img_ndbsi  = raster.read('/home/jupyter-bryan/ISA_Data/Raster/PH_3-Month_NDBSI_2019_2024.tif')
ds_cbi, img_cbi  = raster.read('/home/jupyter-bryan/ISA_Data/Raster/PH_3-Month_CBI_2019_2024.tif')
ds_uci, img_uci  = raster.read('/home/jupyter-bryan/ISA_Data/Raster/PH_3-Month_UCI_2019_2024.tif')

print(img_ndvi.shape)
print(img_mndwi.shape)
print(img_ndbi.shape)
print(img_ndbsi.shape)
print(img_cbi.shape)
print(img_uci.shape)

Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed


(24, 1973, 2560)
(24, 1973, 2560)
(24, 1973, 2560)
(24, 1973, 2560)
(24, 1973, 2560)
(24, 1973, 2560)


Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed


In [10]:
img_ndvi_reshape = img_ndvi.reshape(img_ndvi.shape[0],(img_ndvi.shape[1]*img_ndvi.shape[2])).T
img_mndwi_reshape = img_mndwi.reshape(img_mndwi.shape[0],(img_mndwi.shape[1]*img_mndwi.shape[2])).T
img_ndbi_reshape = img_ndbi.reshape(img_ndbi.shape[0],(img_ndbi.shape[1]*img_ndbi.shape[2])).T
img_ndbsi_reshape = img_ndbsi.reshape(img_ndbsi.shape[0],(img_ndbsi.shape[1]*img_ndbsi.shape[2])).T
img_cbi_reshape = img_cbi.reshape(img_cbi.shape[0],(img_cbi.shape[1]*img_cbi.shape[2])).T
img_uci_reshape = img_uci.reshape(img_uci.shape[0],(img_uci.shape[1]*img_uci.shape[2])).T

In [11]:
def clean_and_interpolate(data):
    df = pd.DataFrame(data, dtype=np.float32) 
    df.dropna(how='all')
    
    if len(df) > 100000:  
        print(f"Processing large dataset with {len(df):,} pixels in chunks...")
        chunk_size = 50000
        results = []
        for i in range(0, len(df), chunk_size):
            chunk = df.iloc[i:i+chunk_size].copy()
            chunk_linear = chunk.interpolate(method='linear', limit_direction='both', axis=1, limit=None)
            chunk_final = chunk_linear.interpolate(method='spline', order=3, axis=1).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)
            chunk_clipped = chunk_final.clip(-1e6, 1e6)
            results.append(chunk_clipped.values.astype(np.float32))
            print(f"Processed chunk {i//chunk_size + 1}/{(len(df)-1)//chunk_size + 1}")
            gc.collect()
        valid_result = np.vstack(results)
    else:
        df_linear = df.interpolate(method='linear', limit_direction='both', axis=1, limit=None)
        df_final = df_linear.interpolate(method='spline', order=3, axis=1).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)
        valid_result = df_final.clip(-1e6, 1e6).values.astype(np.float32)

    return valid_result

In [12]:
%%time
print("\n=== Interpolating NDVI ===")
img_ndvi_int = clean_and_interpolate(img_ndvi_reshape)

print("\n=== Interpolating MNDWI ===")
img_mndwi_int = clean_and_interpolate(img_mndwi_reshape)

print("\n=== Interpolating NDBI ===")
img_ndbi_int = clean_and_interpolate(img_ndbi_reshape)

print("\n=== Interpolating NDBSI ===")
img_ndbsi_int = clean_and_interpolate(img_ndbsi_reshape)

print("\n=== Interpolating CBI ===")
img_cbi_int = clean_and_interpolate(img_cbi_reshape)

print("\n=== Interpolating UCI ===")
img_uci_int = clean_and_interpolate(img_uci_reshape)


=== Interpolating NDVI ===
Processing large dataset with 5,050,880 pixels in chunks...
Processed chunk 1/102
Processed chunk 2/102
Processed chunk 3/102
Processed chunk 4/102
Processed chunk 5/102
Processed chunk 6/102
Processed chunk 7/102
Processed chunk 8/102
Processed chunk 9/102
Processed chunk 10/102
Processed chunk 11/102
Processed chunk 12/102
Processed chunk 13/102
Processed chunk 14/102
Processed chunk 15/102
Processed chunk 16/102
Processed chunk 17/102
Processed chunk 18/102
Processed chunk 19/102
Processed chunk 20/102
Processed chunk 21/102
Processed chunk 22/102
Processed chunk 23/102
Processed chunk 24/102
Processed chunk 25/102
Processed chunk 26/102
Processed chunk 27/102
Processed chunk 28/102
Processed chunk 29/102
Processed chunk 30/102
Processed chunk 31/102
Processed chunk 32/102
Processed chunk 33/102
Processed chunk 34/102
Processed chunk 35/102
Processed chunk 36/102
Processed chunk 37/102
Processed chunk 38/102
Processed chunk 39/102
Processed chunk 40/102
P

In [13]:
# Stack all bands
print("Stacking all bands...")
comb_all = np.stack((img_ndvi_int, img_mndwi_int, 
                     img_ndbi_int, img_ndbsi_int, img_cbi_int, img_uci_int), axis=1)

# Reshape for model input
comb_all_reshape = comb_all.reshape(comb_all.shape[0], comb_all.shape[2], comb_all.shape[1])
print(f"Final input shape: {comb_all_reshape.shape}")

# Clear intermediate variables to free memory
del img_ndvi_int, img_mndwi_int, img_ndbi_int, img_ndbsi_int, img_cbi_int, img_uci_int
del img_ndvi_reshape, img_mndwi_reshape, img_ndbi_reshape, img_ndbsi_reshape, img_cbi_reshape, img_uci_reshape
del comb_all
gc.collect()

Stacking all bands...
Final input shape: (5050880, 24, 6)


0

In [14]:
# BATCH PREDICTION - Process in smaller chunks to avoid memory issues
def predict_in_batches(model, data, batch_size=5000):
    """Predict data in batches to manage memory"""
    n_samples = data.shape[0]
    predictions = []
    
    print(f"Predicting {n_samples:,} pixels in batches of {batch_size:,}...")
    
    for i in range(0, n_samples, batch_size):
        end_idx = min(i + batch_size, n_samples)
        batch = data[i:end_idx]
        batch = np.asarray(batch, dtype=np.float32)
        
        try:
            batch_pred = model.predict(batch, verbose=0)
            predictions.append(batch_pred)
            print(f"Processed batch {i//batch_size + 1}/{(n_samples-1)//batch_size + 1}")
        except Exception as e:
            print(f"Error in batch {i//batch_size + 1}: {e}")
            # Try with smaller batch size
            smaller_batch_size = batch_size // 2
            print(f"Retrying with smaller batch size: {smaller_batch_size}")
            for j in range(i, end_idx, smaller_batch_size):
                small_end = min(j + smaller_batch_size, end_idx)
                small_batch = data[j:small_end]
                small_batch = np.asarray(small_batch, dtype=np.float32)
                small_pred = model.predict(small_batch, verbose=0)
                predictions.append(small_pred)
        
        tf.keras.backend.clear_session()
        gc.collect()
    
    return np.vstack(predictions)

In [15]:
%%time
# Perform prediction
print("Starting prediction...")
try:
    pred_ = predict_in_batches(model, comb_all_reshape, batch_size=2000)
    
except Exception as e:
    print(f"Error during prediction: {e}")
    print("Trying with even smaller batch size...")
    try:
        pred_ = predict_in_batches(model, comb_all_reshape, batch_size=1000)
    except Exception as e2:
        print(f"Still getting error: {e2}")
        print("Trying CPU-only prediction...")
        with tf.device('/CPU:0'):
            pred_ = predict_in_batches(model, comb_all_reshape, batch_size=1000)

print("Processing predictions...")
lc_pred_ = pred_.argmax(axis=1)

# Reshape back to image dimensions
img_pred = np.reshape(lc_pred_, (ds_ndvi.RasterYSize, ds_ndvi.RasterXSize))

print(f"Prediction shape: {img_pred.shape}")
print(f"Prediction min: {img_pred.min()}, max: {img_pred.max()}")
print(f"Unique classes: {np.unique(img_pred)}")

# Export result
outFile = r'/home/jupyter-bryan/ISA_Data/Raster_02/ISA_PH_Multi_Orig_1DConus_FineTune.tif'
raster.export(img_pred, ds_ndvi, filename=outFile, dtype='int')
print(f"Prediction saved to: {outFile}")

Starting prediction...
Predicting 5,050,880 pixels in batches of 2,000...


2025-12-21 18:15:43.431886: I external/local_xla/xla/service/service.cc:163] XLA service 0x794e38004a40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-21 18:15:43.431914: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2025-12-21 18:15:43.452333: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-21 18:15:43.578214: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002
I0000 00:00:1766312145.648510 2869798 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Processed batch 1/2526
Processed batch 2/2526
Processed batch 3/2526
Processed batch 4/2526
Processed batch 5/2526
Processed batch 6/2526
Processed batch 7/2526
Processed batch 8/2526
Processed batch 9/2526
Processed batch 10/2526
Processed batch 11/2526
Processed batch 12/2526
Processed batch 13/2526
Processed batch 14/2526
Processed batch 15/2526
Processed batch 16/2526
Processed batch 17/2526
Processed batch 18/2526
Processed batch 19/2526
Processed batch 20/2526
Processed batch 21/2526
Processed batch 22/2526
Processed batch 23/2526
Processed batch 24/2526
Processed batch 25/2526
Processed batch 26/2526
Processed batch 27/2526
Processed batch 28/2526
Processed batch 29/2526
Processed batch 30/2526
Processed batch 31/2526
Processed batch 32/2526
Processed batch 33/2526
Processed batch 34/2526
Processed batch 35/2526
Processed batch 36/2526
Processed batch 37/2526
Processed batch 38/2526
Processed batch 39/2526
Processed batch 40/2526
Processed batch 41/2526
Processed batch 42/2526
P

ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
